In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torchvision import transforms
from torchvision import models
from torchvision.models.resnet import resnet34
from torchvision.transforms.transforms import Resize
 
transform = transforms.Compose([
transforms.Resize(224),
transforms.ToTensor(),
transforms.Normalize([0.485, 0.456, 0.406],[0.229, 0.224, 0.225])
])
#采用自带的Cifar100
trainset = torchvision.datasets.CIFAR100(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=200, shuffle=True)
 
testset = torchvision.datasets.CIFAR100(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=200, shuffle=False)
net=models.resnet34(weights=True)
##迁移学习
for param in net.parameters(): #固定参数
    print(param.names)
    param.requires_grad = False
 
fc_inputs = net.fc.in_features #获得fc特征层的输入
net.fc = nn.Sequential(         #重新定义特征层，根据需要可以添加自己想要的Linear层
    nn.Linear(fc_inputs, 100),  #多加几层都没关系
    nn.LogSoftmax(dim=1)
)
 
net = net.to('cuda')
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.01, momentum=0.9)
 
# Training
def train(epoch):
    # print('\nEpoch: %d' % epoch)
    net.train()
    train_loss = 0
    correct = 0
    total = 0
    for batch_idx, (inputs, targets) in enumerate(trainloader):
        inputs, targets = inputs.to('cuda'), targets.to('cuda')
        optimizer.zero_grad()
        outputs = net(torch.squeeze(inputs, 1))
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()
 
        print(batch_idx+1,'/', len(trainloader),'epoch: %d' % epoch, '| Loss: %.3f | Acc: %.3f%% (%d/%d)'
            % (train_loss/(batch_idx+1), 100.*correct/total, correct, total))
 
def test(epoch):
    global best_acc
    net.eval()
    test_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(testloader):
            inputs, targets = inputs.to('cuda'), targets.to('cuda')
            outputs = net(torch.squeeze(inputs, 1))
            loss = criterion(outputs, targets)
            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()
            print(batch_idx,'/',len(testloader),'epoch: %d'% epoch, 'Loss: %.3f | Acc: %.3f%% (%d/%d)'
                % (test_loss/(batch_idx+1), 100.*correct/total, correct, total))
 
for epoch in range(50): #设置成100循环
    train(epoch)
torch.save(net.state_dict(),'resnet34cifar100.pkl') #训练完成后保存模型，供下次继续训练使用
print('begin  test ')
 
# for epoch in range(5): #测试5次
#    test(epoch)


Files already downloaded and verified
Files already downloaded and verified


d:\Anaconda3\envs\pytorch\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet34_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet34_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


(None, None, None, None)
(None,)
(None,)
(None, None, None, None)
(None,)
(None,)
(None, None, None, None)
(None,)
(None,)
(None, None, None, None)
(None,)
(None,)
(None, None, None, None)
(None,)
(None,)
(None, None, None, None)
(None,)
(None,)
(None, None, None, None)
(None,)
(None,)
(None, None, None, None)
(None,)
(None,)
(None, None, None, None)
(None,)
(None,)
(None, None, None, None)
(None,)
(None,)
(None, None, None, None)
(None,)
(None,)
(None, None, None, None)
(None,)
(None,)
(None, None, None, None)
(None,)
(None,)
(None, None, None, None)
(None,)
(None,)
(None, None, None, None)
(None,)
(None,)
(None, None, None, None)
(None,)
(None,)
(None, None, None, None)
(None,)
(None,)
(None, None, None, None)
(None,)
(None,)
(None, None, None, None)
(None,)
(None,)
(None, None, None, None)
(None,)
(None,)
(None, None, None, None)
(None,)
(None,)
(None, None, None, None)
(None,)
(None,)
(None, None, None, None)
(None,)
(None,)
(None, None, None, None)
(None,)
(None,)
(None, None, Non

116 / 250 epoch: 0 | Loss: 3.318 | Acc: 28.737% (6667/23200)
117 / 250 epoch: 0 | Loss: 3.310 | Acc: 28.906% (6764/23400)
118 / 250 epoch: 0 | Loss: 3.301 | Acc: 29.055% (6857/23600)
119 / 250 epoch: 0 | Loss: 3.294 | Acc: 29.231% (6957/23800)
120 / 250 epoch: 0 | Loss: 3.284 | Acc: 29.421% (7061/24000)
121 / 250 epoch: 0 | Loss: 3.276 | Acc: 29.574% (7157/24200)
122 / 250 epoch: 0 | Loss: 3.268 | Acc: 29.725% (7253/24400)
123 / 250 epoch: 0 | Loss: 3.259 | Acc: 29.886% (7352/24600)
124 / 250 epoch: 0 | Loss: 3.251 | Acc: 30.032% (7448/24800)
125 / 250 epoch: 0 | Loss: 3.244 | Acc: 30.156% (7539/25000)
126 / 250 epoch: 0 | Loss: 3.237 | Acc: 30.310% (7638/25200)
127 / 250 epoch: 0 | Loss: 3.228 | Acc: 30.492% (7745/25400)
128 / 250 epoch: 0 | Loss: 3.220 | Acc: 30.660% (7849/25600)
129 / 250 epoch: 0 | Loss: 3.213 | Acc: 30.775% (7940/25800)
130 / 250 epoch: 0 | Loss: 3.205 | Acc: 30.888% (8031/26000)
131 / 250 epoch: 0 | Loss: 3.197 | Acc: 31.080% (8143/26200)
132 / 250 epoch: 0 | Los

249 / 250 epoch: 0 | Loss: 2.632 | Acc: 40.719% (20278/49800)
250 / 250 epoch: 0 | Loss: 2.629 | Acc: 40.768% (20384/50000)
1 / 250 epoch: 1 | Loss: 1.870 | Acc: 51.500% (103/200)
2 / 250 epoch: 1 | Loss: 1.787 | Acc: 53.250% (213/400)
3 / 250 epoch: 1 | Loss: 1.757 | Acc: 55.000% (330/600)
4 / 250 epoch: 1 | Loss: 1.789 | Acc: 54.375% (435/800)
5 / 250 epoch: 1 | Loss: 1.775 | Acc: 54.600% (546/1000)
6 / 250 epoch: 1 | Loss: 1.757 | Acc: 55.250% (663/1200)
7 / 250 epoch: 1 | Loss: 1.766 | Acc: 54.857% (768/1400)
8 / 250 epoch: 1 | Loss: 1.759 | Acc: 55.250% (884/1600)
9 / 250 epoch: 1 | Loss: 1.749 | Acc: 55.667% (1002/1800)
10 / 250 epoch: 1 | Loss: 1.745 | Acc: 55.750% (1115/2000)
11 / 250 epoch: 1 | Loss: 1.756 | Acc: 55.500% (1221/2200)
12 / 250 epoch: 1 | Loss: 1.760 | Acc: 55.125% (1323/2400)
13 / 250 epoch: 1 | Loss: 1.758 | Acc: 55.269% (1437/2600)
14 / 250 epoch: 1 | Loss: 1.759 | Acc: 55.143% (1544/2800)
15 / 250 epoch: 1 | Loss: 1.753 | Acc: 55.300% (1659/3000)
16 / 250 epo

...

201 / 250 epoch: 47 | Loss: 0.927 | Acc: 73.052% (29367/40200)
202 / 250 epoch: 47 | Loss: 0.927 | Acc: 73.052% (29513/40400)
203 / 250 epoch: 47 | Loss: 0.928 | Acc: 73.034% (29652/40600)
204 / 250 epoch: 47 | Loss: 0.928 | Acc: 73.007% (29787/40800)
205 / 250 epoch: 47 | Loss: 0.929 | Acc: 73.000% (29930/41000)
206 / 250 epoch: 47 | Loss: 0.929 | Acc: 72.993% (30073/41200)
207 / 250 epoch: 47 | Loss: 0.929 | Acc: 72.995% (30220/41400)
208 / 250 epoch: 47 | Loss: 0.930 | Acc: 72.974% (30357/41600)
209 / 250 epoch: 47 | Loss: 0.930 | Acc: 72.969% (30501/41800)
210 / 250 epoch: 47 | Loss: 0.930 | Acc: 72.974% (30649/42000)
211 / 250 epoch: 47 | Loss: 0.930 | Acc: 72.974% (30795/42200)
212 / 250 epoch: 47 | Loss: 0.929 | Acc: 72.988% (30947/42400)
213 / 250 epoch: 47 | Loss: 0.929 | Acc: 73.002% (31099/42600)
214 / 250 epoch: 47 | Loss: 0.929 | Acc: 73.014% (31250/42800)
215 / 250 epoch: 47 | Loss: 0.929 | Acc: 73.023% (31400/43000)
216 / 250 epoch: 47 | Loss: 0.928 | Acc: 73.030% (31549

85 / 250 epoch: 48 | Loss: 0.913 | Acc: 73.600% (12512/17000)
86 / 250 epoch: 48 | Loss: 0.914 | Acc: 73.564% (12653/17200)
87 / 250 epoch: 48 | Loss: 0.913 | Acc: 73.580% (12803/17400)
88 / 250 epoch: 48 | Loss: 0.913 | Acc: 73.574% (12949/17600)
89 / 250 epoch: 48 | Loss: 0.914 | Acc: 73.539% (13090/17800)
90 / 250 epoch: 48 | Loss: 0.914 | Acc: 73.550% (13239/18000)
91 / 250 epoch: 48 | Loss: 0.913 | Acc: 73.577% (13391/18200)
92 / 250 epoch: 48 | Loss: 0.914 | Acc: 73.571% (13537/18400)
93 / 250 epoch: 48 | Loss: 0.915 | Acc: 73.565% (13683/18600)
94 / 250 epoch: 48 | Loss: 0.915 | Acc: 73.548% (13827/18800)
95 / 250 epoch: 48 | Loss: 0.917 | Acc: 73.468% (13959/19000)
96 / 250 epoch: 48 | Loss: 0.918 | Acc: 73.417% (14096/19200)
97 / 250 epoch: 48 | Loss: 0.918 | Acc: 73.423% (14244/19400)
98 / 250 epoch: 48 | Loss: 0.918 | Acc: 73.459% (14398/19600)
99 / 250 epoch: 48 | Loss: 0.918 | Acc: 73.455% (14544/19800)
100 / 250 epoch: 48 | Loss: 0.921 | Acc: 73.405% (14681/20000)
101 / 2

216 / 250 epoch: 48 | Loss: 0.924 | Acc: 73.269% (31652/43200)
217 / 250 epoch: 48 | Loss: 0.924 | Acc: 73.256% (31793/43400)
218 / 250 epoch: 48 | Loss: 0.924 | Acc: 73.250% (31937/43600)
219 / 250 epoch: 48 | Loss: 0.924 | Acc: 73.253% (32085/43800)
220 / 250 epoch: 48 | Loss: 0.924 | Acc: 73.252% (32231/44000)
221 / 250 epoch: 48 | Loss: 0.923 | Acc: 73.247% (32375/44200)
222 / 250 epoch: 48 | Loss: 0.924 | Acc: 73.234% (32516/44400)
223 / 250 epoch: 48 | Loss: 0.924 | Acc: 73.231% (32661/44600)
224 / 250 epoch: 48 | Loss: 0.925 | Acc: 73.217% (32801/44800)
225 / 250 epoch: 48 | Loss: 0.925 | Acc: 73.209% (32944/45000)
226 / 250 epoch: 48 | Loss: 0.925 | Acc: 73.206% (33089/45200)
227 / 250 epoch: 48 | Loss: 0.926 | Acc: 73.185% (33226/45400)
228 / 250 epoch: 48 | Loss: 0.926 | Acc: 73.173% (33367/45600)
229 / 250 epoch: 48 | Loss: 0.926 | Acc: 73.181% (33517/45800)
230 / 250 epoch: 48 | Loss: 0.926 | Acc: 73.174% (33660/46000)
231 / 250 epoch: 48 | Loss: 0.927 | Acc: 73.136% (33789

100 / 250 epoch: 49 | Loss: 0.916 | Acc: 73.470% (14694/20000)
101 / 250 epoch: 49 | Loss: 0.916 | Acc: 73.465% (14840/20200)
102 / 250 epoch: 49 | Loss: 0.915 | Acc: 73.471% (14988/20400)
103 / 250 epoch: 49 | Loss: 0.916 | Acc: 73.481% (15137/20600)
104 / 250 epoch: 49 | Loss: 0.913 | Acc: 73.558% (15300/20800)
105 / 250 epoch: 49 | Loss: 0.913 | Acc: 73.562% (15448/21000)
106 / 250 epoch: 49 | Loss: 0.914 | Acc: 73.495% (15581/21200)
107 / 250 epoch: 49 | Loss: 0.914 | Acc: 73.495% (15728/21400)
108 / 250 epoch: 49 | Loss: 0.913 | Acc: 73.500% (15876/21600)
109 / 250 epoch: 49 | Loss: 0.913 | Acc: 73.514% (16026/21800)
110 / 250 epoch: 49 | Loss: 0.914 | Acc: 73.495% (16169/22000)
111 / 250 epoch: 49 | Loss: 0.915 | Acc: 73.468% (16310/22200)
112 / 250 epoch: 49 | Loss: 0.915 | Acc: 73.513% (16467/22400)
113 / 250 epoch: 49 | Loss: 0.916 | Acc: 73.434% (16596/22600)
114 / 250 epoch: 49 | Loss: 0.917 | Acc: 73.456% (16748/22800)
115 / 250 epoch: 49 | Loss: 0.917 | Acc: 73.470% (16898

231 / 250 epoch: 49 | Loss: 0.916 | Acc: 73.498% (33956/46200)
232 / 250 epoch: 49 | Loss: 0.916 | Acc: 73.491% (34100/46400)
233 / 250 epoch: 49 | Loss: 0.916 | Acc: 73.483% (34243/46600)
234 / 250 epoch: 49 | Loss: 0.917 | Acc: 73.466% (34382/46800)
235 / 250 epoch: 49 | Loss: 0.917 | Acc: 73.453% (34523/47000)
236 / 250 epoch: 49 | Loss: 0.916 | Acc: 73.468% (34677/47200)
237 / 250 epoch: 49 | Loss: 0.917 | Acc: 73.464% (34822/47400)
238 / 250 epoch: 49 | Loss: 0.916 | Acc: 73.473% (34973/47600)
239 / 250 epoch: 49 | Loss: 0.916 | Acc: 73.462% (35115/47800)
240 / 250 epoch: 49 | Loss: 0.917 | Acc: 73.440% (35251/48000)
241 / 250 epoch: 49 | Loss: 0.918 | Acc: 73.432% (35394/48200)
242 / 250 epoch: 49 | Loss: 0.918 | Acc: 73.442% (35546/48400)
243 / 250 epoch: 49 | Loss: 0.918 | Acc: 73.434% (35689/48600)
244 / 250 epoch: 49 | Loss: 0.918 | Acc: 73.441% (35839/48800)
245 / 250 epoch: 49 | Loss: 0.918 | Acc: 73.427% (35979/49000)
246 / 250 epoch: 49 | Loss: 0.918 | Acc: 73.411% (36118

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torchvision import transforms
from torchvision import models
from torchvision.models.resnet import resnet34
from torchvision.transforms.transforms import Resize
 
transform = transforms.Compose([
transforms.Resize(224),
transforms.ToTensor(),
transforms.Normalize([0.485, 0.456, 0.406],[0.229, 0.224, 0.225]) #默认的标准化参数
])
 
 
trainset = torchvision.datasets.CIFAR100(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=100, shuffle=True)
 
testset = torchvision.datasets.CIFAR100(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=100, shuffle=False)
 
net=models.resnet34(num_classes = 100) # pretrained=False or True 不重要
fc_inputs = net.fc.in_features # 保持与前面第一步中的代码一致
net.fc = nn.Sequential(         #
    nn.Linear(fc_inputs, 100),  #
    nn.LogSoftmax(dim=1)
)
 
net.load_state_dict(torch.load('resnet34cifar100.pkl')) #装载上传训练的参数
mydict=net.state_dict()
#for k,v in mydict.items():    
#    print('k===',k,'||||,v==',v)
 
models=net.modules()
for p in models:
    if p._get_name()!='Linear':
        print(p._get_name())
        p.requires_grad_=False
 
net = net.to('cuda')
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9) #减小 lr
 
# Training
def train(epoch):
    # print('\nEpoch: %d' % epoch)
    net.train()
    train_loss = 0
    correct = 0
    total = 0
    for batch_idx, (inputs, targets) in enumerate(trainloader):
        inputs, targets = inputs.to('cuda'), targets.to('cuda')
        optimizer.zero_grad()
        outputs = net(torch.squeeze(inputs, 1))
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
 
        train_loss += loss.item()
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()
 
        print(batch_idx+1,'/', len(trainloader),'epoch: %d' % epoch, '| Loss: %.3f | Acc: %.3f%% (%d/%d)'
            % (train_loss/(batch_idx+1), 100.*correct/total, correct, total))
 
def test(epoch):
    global best_acc
    net.eval()
    test_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(testloader):
            inputs, targets = inputs.to('cuda'), targets.to('cuda')
            outputs = net(torch.squeeze(inputs, 1))
            loss = criterion(outputs, targets)
 
            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()
 
            print(batch_idx,'/',len(testloader),'epoch: %d'% epoch, 'Loss: %.3f | Acc: %.3f%% (%d/%d)'
                % (test_loss/(batch_idx+1), 100.*correct/total, correct, total))
 
 
for epoch in range(20):
    train(epoch)
torch.save(net.state_dict(),'resnet34cifar100.pkl') #训练完成后保存模型，供下次继续训练使用
 
print('begin  test ')
#for epoch in range(5):
#    test(epoch)


Files already downloaded and verified
Files already downloaded and verified
ResNet
Conv2d
BatchNorm2d
ReLU
MaxPool2d
Sequential
BasicBlock
Conv2d
BatchNorm2d
ReLU
Conv2d
BatchNorm2d
BasicBlock
Conv2d
BatchNorm2d
ReLU
Conv2d
BatchNorm2d
BasicBlock
Conv2d
BatchNorm2d
ReLU
Conv2d
BatchNorm2d
Sequential
BasicBlock
Conv2d
BatchNorm2d
ReLU
Conv2d
BatchNorm2d
Sequential
Conv2d
BatchNorm2d
BasicBlock
Conv2d
BatchNorm2d
ReLU
Conv2d
BatchNorm2d
BasicBlock
Conv2d
BatchNorm2d
ReLU
Conv2d
BatchNorm2d
BasicBlock
Conv2d
BatchNorm2d
ReLU
Conv2d
BatchNorm2d
Sequential
BasicBlock
Conv2d
BatchNorm2d
ReLU
Conv2d
BatchNorm2d
Sequential
Conv2d
BatchNorm2d
BasicBlock
Conv2d
BatchNorm2d
ReLU
Conv2d
BatchNorm2d
BasicBlock
Conv2d
BatchNorm2d
ReLU
Conv2d
BatchNorm2d
BasicBlock
Conv2d
BatchNorm2d
ReLU
Conv2d
BatchNorm2d
BasicBlock
Conv2d
BatchNorm2d
ReLU
Conv2d
BatchNorm2d
BasicBlock
Conv2d
BatchNorm2d
ReLU
Conv2d
BatchNorm2d
Sequential
BasicBlock
Conv2d
BatchNorm2d
ReLU
Conv2d
BatchNorm2d
Sequential
Conv2d
Batch

121 / 500 epoch: 0 | Loss: 0.976 | Acc: 71.479% (8649/12100)
122 / 500 epoch: 0 | Loss: 0.975 | Acc: 71.516% (8725/12200)
123 / 500 epoch: 0 | Loss: 0.973 | Acc: 71.528% (8798/12300)
124 / 500 epoch: 0 | Loss: 0.971 | Acc: 71.573% (8875/12400)
125 / 500 epoch: 0 | Loss: 0.972 | Acc: 71.552% (8944/12500)
126 / 500 epoch: 0 | Loss: 0.970 | Acc: 71.571% (9018/12600)
127 / 500 epoch: 0 | Loss: 0.968 | Acc: 71.638% (9098/12700)
128 / 500 epoch: 0 | Loss: 0.969 | Acc: 71.602% (9165/12800)
129 / 500 epoch: 0 | Loss: 0.966 | Acc: 71.674% (9246/12900)
130 / 500 epoch: 0 | Loss: 0.964 | Acc: 71.731% (9325/13000)
131 / 500 epoch: 0 | Loss: 0.962 | Acc: 71.771% (9402/13100)
132 / 500 epoch: 0 | Loss: 0.962 | Acc: 71.765% (9473/13200)
133 / 500 epoch: 0 | Loss: 0.962 | Acc: 71.782% (9547/13300)
134 / 500 epoch: 0 | Loss: 0.963 | Acc: 71.791% (9620/13400)
135 / 500 epoch: 0 | Loss: 0.961 | Acc: 71.830% (9697/13500)
136 / 500 epoch: 0 | Loss: 0.959 | Acc: 71.890% (9777/13600)
137 / 500 epoch: 0 | Los

254 / 500 epoch: 0 | Loss: 0.855 | Acc: 74.618% (18953/25400)
255 / 500 epoch: 0 | Loss: 0.855 | Acc: 74.631% (19031/25500)
256 / 500 epoch: 0 | Loss: 0.855 | Acc: 74.648% (19110/25600)
257 / 500 epoch: 0 | Loss: 0.855 | Acc: 74.650% (19185/25700)
258 / 500 epoch: 0 | Loss: 0.855 | Acc: 74.647% (19259/25800)
259 / 500 epoch: 0 | Loss: 0.855 | Acc: 74.629% (19329/25900)
260 / 500 epoch: 0 | Loss: 0.854 | Acc: 74.627% (19403/26000)
261 / 500 epoch: 0 | Loss: 0.853 | Acc: 74.663% (19487/26100)
262 / 500 epoch: 0 | Loss: 0.852 | Acc: 74.691% (19569/26200)
263 / 500 epoch: 0 | Loss: 0.851 | Acc: 74.726% (19653/26300)
264 / 500 epoch: 0 | Loss: 0.851 | Acc: 74.754% (19735/26400)
265 / 500 epoch: 0 | Loss: 0.850 | Acc: 74.758% (19811/26500)
266 / 500 epoch: 0 | Loss: 0.851 | Acc: 74.748% (19883/26600)
267 / 500 epoch: 0 | Loss: 0.850 | Acc: 74.749% (19958/26700)
268 / 500 epoch: 0 | Loss: 0.850 | Acc: 74.761% (20036/26800)
269 / 500 epoch: 0 | Loss: 0.849 | Acc: 74.792% (20119/26900)
270 / 50

387 / 500 epoch: 0 | Loss: 0.805 | Acc: 75.788% (29330/38700)
388 / 500 epoch: 0 | Loss: 0.805 | Acc: 75.804% (29412/38800)
389 / 500 epoch: 0 | Loss: 0.804 | Acc: 75.817% (29493/38900)
390 / 500 epoch: 0 | Loss: 0.804 | Acc: 75.833% (29575/39000)
391 / 500 epoch: 0 | Loss: 0.804 | Acc: 75.839% (29653/39100)
392 / 500 epoch: 0 | Loss: 0.803 | Acc: 75.852% (29734/39200)
393 / 500 epoch: 0 | Loss: 0.803 | Acc: 75.855% (29811/39300)
394 / 500 epoch: 0 | Loss: 0.802 | Acc: 75.873% (29894/39400)
395 / 500 epoch: 0 | Loss: 0.802 | Acc: 75.891% (29977/39500)
396 / 500 epoch: 0 | Loss: 0.801 | Acc: 75.907% (30059/39600)
397 / 500 epoch: 0 | Loss: 0.801 | Acc: 75.904% (30134/39700)
398 / 500 epoch: 0 | Loss: 0.801 | Acc: 75.902% (30209/39800)
399 / 500 epoch: 0 | Loss: 0.801 | Acc: 75.907% (30287/39900)
400 / 500 epoch: 0 | Loss: 0.801 | Acc: 75.920% (30368/40000)
401 / 500 epoch: 0 | Loss: 0.801 | Acc: 75.928% (30447/40100)
402 / 500 epoch: 0 | Loss: 0.801 | Acc: 75.918% (30519/40200)
403 / 50

21 / 500 epoch: 1 | Loss: 0.387 | Acc: 87.810% (1844/2100)
22 / 500 epoch: 1 | Loss: 0.387 | Acc: 88.000% (1936/2200)
23 / 500 epoch: 1 | Loss: 0.383 | Acc: 88.174% (2028/2300)
24 / 500 epoch: 1 | Loss: 0.383 | Acc: 88.292% (2119/2400)
25 / 500 epoch: 1 | Loss: 0.382 | Acc: 88.280% (2207/2500)
26 / 500 epoch: 1 | Loss: 0.381 | Acc: 88.231% (2294/2600)
27 / 500 epoch: 1 | Loss: 0.379 | Acc: 88.407% (2387/2700)
28 / 500 epoch: 1 | Loss: 0.379 | Acc: 88.429% (2476/2800)
29 / 500 epoch: 1 | Loss: 0.378 | Acc: 88.448% (2565/2900)
30 / 500 epoch: 1 | Loss: 0.376 | Acc: 88.600% (2658/3000)
31 / 500 epoch: 1 | Loss: 0.371 | Acc: 88.710% (2750/3100)
32 / 500 epoch: 1 | Loss: 0.368 | Acc: 88.812% (2842/3200)
33 / 500 epoch: 1 | Loss: 0.366 | Acc: 88.788% (2930/3300)
34 / 500 epoch: 1 | Loss: 0.363 | Acc: 88.853% (3021/3400)
35 / 500 epoch: 1 | Loss: 0.362 | Acc: 88.943% (3113/3500)
36 / 500 epoch: 1 | Loss: 0.365 | Acc: 88.778% (3196/3600)
37 / 500 epoch: 1 | Loss: 0.365 | Acc: 88.865% (3288/370

...

436 / 500 epoch: 18 | Loss: 0.006 | Acc: 99.975% (43589/43600)
437 / 500 epoch: 18 | Loss: 0.006 | Acc: 99.975% (43689/43700)
438 / 500 epoch: 18 | Loss: 0.006 | Acc: 99.975% (43789/43800)
439 / 500 epoch: 18 | Loss: 0.006 | Acc: 99.975% (43889/43900)
440 / 500 epoch: 18 | Loss: 0.006 | Acc: 99.975% (43989/44000)
441 / 500 epoch: 18 | Loss: 0.006 | Acc: 99.973% (44088/44100)
442 / 500 epoch: 18 | Loss: 0.006 | Acc: 99.973% (44188/44200)
443 / 500 epoch: 18 | Loss: 0.006 | Acc: 99.973% (44288/44300)
444 / 500 epoch: 18 | Loss: 0.006 | Acc: 99.973% (44388/44400)
445 / 500 epoch: 18 | Loss: 0.006 | Acc: 99.973% (44488/44500)
446 / 500 epoch: 18 | Loss: 0.006 | Acc: 99.973% (44588/44600)
447 / 500 epoch: 18 | Loss: 0.006 | Acc: 99.973% (44688/44700)
448 / 500 epoch: 18 | Loss: 0.006 | Acc: 99.973% (44788/44800)
449 / 500 epoch: 18 | Loss: 0.006 | Acc: 99.973% (44888/44900)
450 / 500 epoch: 18 | Loss: 0.006 | Acc: 99.973% (44988/45000)
451 / 500 epoch: 18 | Loss: 0.006 | Acc: 99.973% (45088

70 / 500 epoch: 19 | Loss: 0.006 | Acc: 99.971% (6998/7000)
71 / 500 epoch: 19 | Loss: 0.006 | Acc: 99.972% (7098/7100)
72 / 500 epoch: 19 | Loss: 0.006 | Acc: 99.972% (7198/7200)
73 / 500 epoch: 19 | Loss: 0.006 | Acc: 99.973% (7298/7300)
74 / 500 epoch: 19 | Loss: 0.006 | Acc: 99.973% (7398/7400)
75 / 500 epoch: 19 | Loss: 0.006 | Acc: 99.973% (7498/7500)
76 / 500 epoch: 19 | Loss: 0.006 | Acc: 99.974% (7598/7600)
77 / 500 epoch: 19 | Loss: 0.006 | Acc: 99.974% (7698/7700)
78 / 500 epoch: 19 | Loss: 0.006 | Acc: 99.974% (7798/7800)
79 / 500 epoch: 19 | Loss: 0.006 | Acc: 99.975% (7898/7900)
80 / 500 epoch: 19 | Loss: 0.006 | Acc: 99.975% (7998/8000)
81 / 500 epoch: 19 | Loss: 0.006 | Acc: 99.975% (8098/8100)
82 / 500 epoch: 19 | Loss: 0.006 | Acc: 99.976% (8198/8200)
83 / 500 epoch: 19 | Loss: 0.006 | Acc: 99.976% (8298/8300)
84 / 500 epoch: 19 | Loss: 0.006 | Acc: 99.976% (8398/8400)
85 / 500 epoch: 19 | Loss: 0.006 | Acc: 99.965% (8497/8500)
86 / 500 epoch: 19 | Loss: 0.006 | Acc: 

202 / 500 epoch: 19 | Loss: 0.006 | Acc: 99.970% (20194/20200)
203 / 500 epoch: 19 | Loss: 0.006 | Acc: 99.970% (20294/20300)
204 / 500 epoch: 19 | Loss: 0.006 | Acc: 99.971% (20394/20400)
205 / 500 epoch: 19 | Loss: 0.006 | Acc: 99.971% (20494/20500)
206 / 500 epoch: 19 | Loss: 0.006 | Acc: 99.966% (20593/20600)
207 / 500 epoch: 19 | Loss: 0.006 | Acc: 99.966% (20693/20700)
208 / 500 epoch: 19 | Loss: 0.006 | Acc: 99.966% (20793/20800)
209 / 500 epoch: 19 | Loss: 0.006 | Acc: 99.967% (20893/20900)
210 / 500 epoch: 19 | Loss: 0.006 | Acc: 99.967% (20993/21000)
211 / 500 epoch: 19 | Loss: 0.006 | Acc: 99.967% (21093/21100)
212 / 500 epoch: 19 | Loss: 0.006 | Acc: 99.967% (21193/21200)
213 / 500 epoch: 19 | Loss: 0.006 | Acc: 99.967% (21293/21300)
214 / 500 epoch: 19 | Loss: 0.006 | Acc: 99.967% (21393/21400)
215 / 500 epoch: 19 | Loss: 0.006 | Acc: 99.967% (21493/21500)
216 / 500 epoch: 19 | Loss: 0.006 | Acc: 99.968% (21593/21600)
217 / 500 epoch: 19 | Loss: 0.006 | Acc: 99.968% (21693

333 / 500 epoch: 19 | Loss: 0.006 | Acc: 99.967% (33289/33300)
334 / 500 epoch: 19 | Loss: 0.006 | Acc: 99.967% (33389/33400)
335 / 500 epoch: 19 | Loss: 0.006 | Acc: 99.967% (33489/33500)
336 / 500 epoch: 19 | Loss: 0.006 | Acc: 99.967% (33589/33600)
337 / 500 epoch: 19 | Loss: 0.006 | Acc: 99.967% (33689/33700)
338 / 500 epoch: 19 | Loss: 0.006 | Acc: 99.967% (33789/33800)
339 / 500 epoch: 19 | Loss: 0.006 | Acc: 99.968% (33889/33900)
340 / 500 epoch: 19 | Loss: 0.006 | Acc: 99.968% (33989/34000)
341 / 500 epoch: 19 | Loss: 0.006 | Acc: 99.968% (34089/34100)
342 / 500 epoch: 19 | Loss: 0.006 | Acc: 99.968% (34189/34200)
343 / 500 epoch: 19 | Loss: 0.006 | Acc: 99.968% (34289/34300)
344 / 500 epoch: 19 | Loss: 0.006 | Acc: 99.968% (34389/34400)
345 / 500 epoch: 19 | Loss: 0.006 | Acc: 99.968% (34489/34500)
346 / 500 epoch: 19 | Loss: 0.006 | Acc: 99.968% (34589/34600)
347 / 500 epoch: 19 | Loss: 0.006 | Acc: 99.968% (34689/34700)
348 / 500 epoch: 19 | Loss: 0.006 | Acc: 99.968% (34789

464 / 500 epoch: 19 | Loss: 0.006 | Acc: 99.972% (46387/46400)
465 / 500 epoch: 19 | Loss: 0.006 | Acc: 99.972% (46487/46500)
466 / 500 epoch: 19 | Loss: 0.006 | Acc: 99.972% (46587/46600)
467 / 500 epoch: 19 | Loss: 0.006 | Acc: 99.972% (46687/46700)
468 / 500 epoch: 19 | Loss: 0.006 | Acc: 99.972% (46787/46800)
469 / 500 epoch: 19 | Loss: 0.006 | Acc: 99.972% (46887/46900)
470 / 500 epoch: 19 | Loss: 0.006 | Acc: 99.972% (46987/47000)
471 / 500 epoch: 19 | Loss: 0.006 | Acc: 99.972% (47087/47100)
472 / 500 epoch: 19 | Loss: 0.006 | Acc: 99.972% (47187/47200)
473 / 500 epoch: 19 | Loss: 0.006 | Acc: 99.973% (47287/47300)
474 / 500 epoch: 19 | Loss: 0.006 | Acc: 99.973% (47387/47400)
475 / 500 epoch: 19 | Loss: 0.006 | Acc: 99.973% (47487/47500)
476 / 500 epoch: 19 | Loss: 0.006 | Acc: 99.973% (47587/47600)
477 / 500 epoch: 19 | Loss: 0.006 | Acc: 99.973% (47687/47700)
478 / 500 epoch: 19 | Loss: 0.006 | Acc: 99.973% (47787/47800)
479 / 500 epoch: 19 | Loss: 0.006 | Acc: 99.973% (47887